In [8]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import sys
import math
import json
import re

hf = "/home"
sys.path.append(hf+"/will/projects/legoproj")
sys.path.append(hf+"/will/projects/training/dataprep")

import feature_utils as fu
from cvscripts import cv_utils as cvu

%matplotlib inline

import tensorflow as tf

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import Session

config = ConfigProto()
config.gpu_options.allow_growth = True
session = Session(config=config)

import keras
from keras.models import load_model


def getObjFromHue(hue,dset):
    hue = int(round(hue/5))
    name = dset["ids"][str(hue)]
    return name

In [25]:
datapath = "/home/will/projects/legoproj/data/kpts_dset_5/"
data = fu.dictFromJson(datapath + "dset.json")
i = 65
tag = "{:0>4}".format(i)
maskpath = datapath + "{}_masks.png".format(tag)

mask = cv2.imread(maskpath)
masks = cvu.separate(mask)

for hue in masks:
    objname = getObjFromHue(hue,data)
    if objname:
        objclass = objname.split(".")[0]
        if objclass == "WingL":
            wing = objname
            break
            

studs = fu.get_object_studs(objclass)
for stud in studs:
    stud.append(1.0)
    
projmat = fu.matrix_from_string(data["projection"])
modelmat = fu.matrix_from_string(data["objects"][objname]["modelmat"])
viewmat = fu.matrix_from_string(data["viewmats"][i])

screenverts = fu.verts_to_screen(modelmat, viewmat, projmat, studs, filter=False)


screenverts[:,0:2] = fu.toNDC(screenverts[:,0:2], (512,512))

overts = np.array(studs, dtype=np.float32)[:,0:3]
overts[:,2] = 0.0
sverts = np.array(screenverts, dtype=np.float32)[:,0:2]
    
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera([overts.astype('float32')], [sverts.astype('float32')], (512,512), None, None)

print(mtx)

#[[576.25736016   0.         248.52187883]
# [  0.         574.53464108 245.49564339]
# [  0.           0.           1.        ]]

[[169.71852943   0.         256.80383098]
 [  0.         115.33817545 260.3110781 ]
 [  0.           0.           1.        ]]


In [23]:
model = load_model("/home/will/projects/legoproj/nets/tstgeom_poleeng_bayes_fr.h5",compile=False)

i = 65
tag = "{:0>4}".format(i)

img = cv2.imread(datapath + "{}_a.png".format(tag),0)        
img = cv2.resize(img,(256,256),interpolation=cv2.INTER_LINEAR)
geomraw = cv2.imread(datapath + "geom/{}_geom.png".format(tag))

pred = model.predict( np.reshape(img, (1,256,256,1)).astype('float32')/255.0 )
pred = (255.0 * np.reshape((1.0+pred)/2.0, (256,256,4))).astype(np.uint8)

geom_pred = pred[:,:,0:3].astype(np.uint8)
confidence = (255 - np.reshape( pred[:,:,3:], (256,256) )).astype(np.uint8)


names = []

maskpath = datapath + "{}_masks.png".format(tag)

mask = cv2.imread(maskpath)
masks = cvu.separate(mask)

for hue in masks:
    objname = getObjFromHue(hue,data)
    #print(objname)
    if objname:
        objclass = objname.split(".")[0]
        if objclass and (objclass == "Engine" or objclass == "Pole"):
            names.append(tuple([objname,hue]))
            
rows = np.arange(256)
cols = np.arange(256)

r,c = np.meshgrid(rows,cols)
inds = np.stack((c,r),axis=-1).astype(np.uint8)
            
for obj in names:
    
    name, hue = obj[0], obj[1]
    objmask = masks[hue]
    
    print(name)
    
    objmask = cv2.resize(objmask, (256,256), cv2.INTER_LINEAR)
    mask = cv2.resize(mask, (256,256), cv2.INTER_LINEAR)
    geom_pred = cv2.resize(geom_pred, (256,256), cv2.INTER_LINEAR)
    confidence = cv2.resize(confidence, (256,256), cv2.INTER_LINEAR)
    
    #region = cv2.bitwise_and(geom_pred,geom_pred,mask=objmask)
    conf = cv2.bitwise_and(confidence,confidence,mask=objmask)
    
    thresh = conf > 240
    
    coords = geom_pred[thresh]/255
    locs = 2 * inds[thresh]
        
    print(locs)
    print(coords)
    print(coords.dtype)

    
    
    
    
    break

Pole.005
[[134 228]
 [134 230]
 [134 232]
 [136 226]
 [136 228]
 [136 230]
 [136 232]
 [136 234]
 [138 224]
 [138 226]
 [138 228]
 [138 230]
 [138 232]
 [138 234]
 [140 224]
 [140 226]
 [140 228]
 [140 230]
 [140 232]
 [140 234]
 [142 224]
 [142 228]
 [142 230]
 [142 232]
 [142 234]
 [144 228]
 [144 230]
 [144 232]
 [144 234]
 [146 228]
 [146 230]
 [146 232]
 [146 234]
 [148 228]
 [148 230]
 [148 232]
 [148 234]
 [150 230]
 [150 232]
 [154 234]
 [156 234]
 [160 236]
 [162 236]
 [168 238]
 [168 240]
 [168 242]
 [170 236]
 [170 238]
 [170 240]
 [170 242]
 [170 244]
 [172 234]
 [172 236]
 [172 238]
 [172 240]
 [172 242]
 [172 244]
 [174 232]
 [174 234]
 [174 236]
 [174 238]
 [174 240]
 [174 242]
 [174 244]
 [174 246]
 [176 232]
 [176 234]
 [176 236]
 [176 238]
 [176 240]
 [176 242]
 [176 244]
 [176 246]
 [178 234]
 [178 236]
 [178 238]
 [178 240]
 [178 242]
 [178 244]
 [178 246]
 [180 234]
 [180 236]
 [180 238]
 [180 240]
 [180 244]
 [180 246]
 [182 238]
 [192 236]]
[[0.86666667 0.0862745

In [ ]:
ret, rvec, tvec,inliers = cv2.solvePnPRansac(obj_pts_s, img_pts_s, mtx,None,flags=cv2.SOLVEPNP_EPNP,reprojectionError=3,iterationsCount=100)

In [ ]:
'''
dist_coeffs = np.zeros((4,1))
ocoordscw = np.reshape(ocoordscw, (4,1,3))
scoordscw = np.reshape(scoordscw, (4,1,2))
ocoordscw = ocoordscw.astype(np.float32)
scoordscw = scoordscw.astype(np.float32)

#ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(ocoordscw, scoordscw, (512, 512),None,None)
r = cv2.solvePnPRansac(ocoordscw, scoordscw, mtx, dist, flags=(cv2.SOLVEPNP_ITERATIVE))

r'''

In [ ]:
rvec = r[1]
tvec = r[2]
tvec